In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import random
import copy

# Dataset Preprocessing

In [152]:
raw_train = pd.read_parquet(r'C:\Users\RACYM\Desktop\fac\M2_PFE\datasets\CIC-DDoS2019\Syn-training.parquet', engine='fastparquet')
raw_test = pd.read_parquet(r'C:\Users\RACYM\Desktop\fac\M2_PFE\datasets\CIC-DDoS2019\Syn-testing.parquet', engine='fastparquet')

In [153]:
raw_test

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,52,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Syn
1,6,999842,4,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Syn
2,6,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Syn
3,6,47,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Syn
4,6,93489757,14,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,1.0,0.0,1.0,1.0,15581614.0,3107965.0,19998236.0,11493926.0,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,6,499664,4,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
903,6,217,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
904,6,194,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
905,17,20808,2,2,66.0,122.0,33.0,33.0,33.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [154]:
print(raw_train.Label)
raw_train.Label = raw_train.Label.map({'Benign': 0, 'Syn': 1})
raw_test.Label = raw_test.Label.map({'Benign': 0, 'Syn': 1})

print(raw_train.Label)

test_set = raw_test.to_numpy()
dataset = raw_train.to_numpy()
dataset[77]
dataset.shape



0           Syn
1           Syn
2           Syn
3           Syn
4           Syn
          ...  
70331    Benign
70332    Benign
70333    Benign
70334    Benign
70335    Benign
Name: Label, Length: 70336, dtype: category
Categories (2, object): ['Benign', 'Syn']
0        1
1        1
2        1
3        1
4        1
        ..
70331    0
70332    0
70333    0
70334    0
70335    0
Name: Label, Length: 70336, dtype: category
Categories (2, int64): [0, 1]


(70336, 78)

In [155]:
type(dataset)


numpy.ndarray

In [156]:
MINs = np.array([0 for i in range(len(dataset[0])-1)])
MAXs = np.array([0 for i in range(len(dataset[0])-1)])


for i in range(len(dataset[:1000])):
    for j in range(len(dataset[0][:-1])):
        if dataset[i][j] < MINs[j]:
            MINs[j] = dataset[i][j]
        if dataset[i][j] > MAXs[j]:
            MAXs[j] = dataset[i][j]

# Algorithm Part

In [157]:
class AIRS:
    """
    :param train_set: training dataset
    :param CLONAL_RATE: the number of ARB clones that will be generated from MC best match
    :param HYPER_CLONAL_RATE: factor to multiply the clonal rate with
    :param MC_INIT_RATE: the percentage of dataset that will become memory cell
    :param MUTATION_RATE: the chance of mutation of each feature
    :param TOTAL_RESSOURCES: the ressource limit
    :param MAX_ITER: max iteration for competing for limited ressources (more iteration -> more ARB mutation)
    :param AFFINITY_THRESHOLD_SCALAR: multiply with the affinity threshold, limit to see if new MC is too close to best match
    :param KNN_K: K value for KNN classification
    """
    def __init__(self, train_set:np.ndarray, CLONAL_RATE:int, HYPER_CLONAL_RATE:int, CLASS_NUMBER:int, MC_INIT_RATE:float,
          MUTATION_RATE:float, TOTAL_RESSOURCES:int, MAX_ITER:int, AFFINITY_THRESHOLD_SCALAR:float, KNN_K:int):
        self.train_set = train_set
        self.CLONAL_RATE = CLONAL_RATE
        self.HYPER_CLONAL_RATE = HYPER_CLONAL_RATE
        self.CLASS_NUMBER = CLASS_NUMBER
        self.MC_INIT_RATE = MC_INIT_RATE
        self.MUTATION_RATE = MUTATION_RATE
        self.TOTAL_RESSOURCES = TOTAL_RESSOURCES
        self.MAX_ITER = MAX_ITER
        self.AFFINITY_THRESHOLD_SCALAR = AFFINITY_THRESHOLD_SCALAR
        self.KNN_K = KNN_K

        self.MC_POOL = {_class : [] for _class in range(CLASS_NUMBER)}
        self.ARB_POOL = {_class : [] for _class in range(CLASS_NUMBER)}
        self.MC_iterations = []

        self.MINs = np.array([0 for i in range(65)])
        self.MAXs = np.array([0 for i in range(65)])
        
        self.AFFINITY_THRESHOLD = 0.0



    @staticmethod
    def affinity(vec1 : np.ndarray, vec2 : np.ndarray) -> float:
        return 1 / (1 + np.linalg.norm(vec1 - vec2))  # 1 / (1 + √Σ(a_i - b_i)²)

        # can also try : np.sqrt(np.sum((vec1 - vec2) ** 2))
        # can calculate ein 2 ways :
        # uclidistance / euclidistance + 1 : makes the distance range between 0-1 (0 close / 1 far)
        # 1 / 1 + euclidistance            : inverse euclidistance (0 low affinity (far) / 1 high affinity (close))


    # Calculating the affinity threshold (average affinity between all vectors)
    # Aproximate methode
    def affinity_threshold_sampled(self, sampleSize:int) -> float:
        aff_threshold = 0.0
        N = self.train_set.shape[0] # gets the number of rows in dataset
        for i in range(sampleSize):
            i_vec1, i_vec2 = random.sample(range(N), k=2)
            aff_threshold += self.affinity(self.train_set[i_vec1][:-1], self.train_set[i_vec2][:-1])
        return aff_threshold / sampleSize

    def affinity_threshold_sampled_average(self, sampleSize:int, repeatCount:int):
        aff_threshold_avg = 0.0
        aff_tracker = 0
        for _ in range(repeatCount):
            aff_tracker += 1
            if aff_tracker % int(repeatCount/10) == 0:
                print('progresion : {} %'.format(int(aff_tracker / repeatCount *100)))
            aff_threshold_avg += self.affinity_threshold_sampled(sampleSize)
        return aff_threshold_avg

    # tests for different repeat values (sample size : 70000)
    # repeated : 10  -> aff_threshold : 8.233312076766814e-06
    # repeated : 20  -> aff_threshold : 8.344028812104138e-06
    # repeated : 30  -> aff_threshold : 9.686835498847982e-06
    # repeated : 40  -> aff_threshold : 8.89222841789201e-06
    # repeated : 50  -> aff_threshold : 9.00554816736841e-06
    # repeated : 100  -> aff_threshold : 9.293699959625458e-06
    # repeated : 500  -> aff_threshold : 9.023340651123559e-06
    # repeated : 1000  -> aff_threshold : 9.099114421126247e-06

    # deterministic methode (not used, too slow)
    def affinity_threshold_deterministic(self) -> float:
        aff_threshold = 0.0
        nb_samples = len(self.train_set)
        for i in range(nb_samples):
            for j in range(i+1, nb_samples):
                aff_threshold = self.affinity(self.train_set[i], self.train_set[j])
        return aff_threshold / nb_samples

    def calculate_affinity_threshold(self):
        print("calculating the affinity threshold...")
        self.AFFINITY_THRESHOLD = self.affinity_threshold_sampled_average(7000, 1000)
        print('Affinity threshold calculated : {}\n'.format(self.AFFINITY_THRESHOLD))




    # MC_INIT_RATE is the percentage of the dataset that will be used to initialize the MC pool
    def init_MC_pool(self):
        for _ in range(int(len(self.train_set) * self.MC_INIT_RATE)):
            random_sample = self.train_set[np.random.randint(len(self.train_set))]
            self.MC_POOL[random_sample[-1]].append(MC(vector=random_sample[:-1], _class=random_sample[-1]))


    def min_res_ARB(self, _class:int):
        minRes = 1.0
        for i in range(len(self.ARB_POOL[_class])):
            if self.ARB_POOL[_class][i].ressources <= minRes:
                minRes = self.ARB_POOL[_class][i].ressources
                minResARB = self.ARB_POOL[_class][i]
                index_ARB = i
        return minResARB, index_ARB

    def get_max_stim_ARB_as_MC(self, _class:int):
        maxStim = 0.0
        for i in range(len(self.ARB_POOL[_class])):
            if self.ARB_POOL[_class][i].stimulation >= maxStim:
                maxStim = self.ARB_POOL[_class][i].stimulation
                maxStimARB = self.ARB_POOL[_class][i]
        return MC(vector=maxStimARB.vector, _class=maxStimARB._class)

    def get_max_stim_MC(self, _class:int):
        maxStim = 0.0
        for i in range(len(self.MC_POOL[_class])):
            if self.MC_POOL[_class][i].stimulation >= maxStim:
                maxStim = self.MC_POOL[_class][i].stimulation
                maxStimMC = self.MC_POOL[_class][i]
        return maxStimMC


    def Min_Max_Stim_ARB(self, pattern:np.ndarray):
        MIN_STIM, MAX_STIM = 0.0, 1.0
        for c in self.ARB_POOL.keys():
            for ARB_cell in self.ARB_POOL.get(c):
                stim = ARB_cell.stimulate(pattern)
                if stim < MIN_STIM:
                    MIN_STIM = stim
                if stim > MAX_STIM:
                    MAX_STIM = stim
        # print('IN MIN MAX STIM FUNCTION : min={}, max={}'.format(MIN_STIM, MAX_STIM)) useless function ??
        return MIN_STIM, MAX_STIM

    def find_MIN_MAX_dataset(self):
        MINs = np.array([0 for i in range(len(self.train_set[0])-1)])
        MAXs = np.array([0 for i in range(len(self.train_set[0])-1)])

        for i in range(len(self.train_set)):
            for j in range(len(self.train_set[0][:-1])):
                if self.train_set[i][j] < MINs[j]:
                    MINs[j] = self.train_set[i][j]
                if self.train_set[i][j] > MAXs[j]:
                    MAXs[j] = self.train_set[i][j]
        return MINs, MAXs





    @staticmethod
    def updateTracker(tracker_count:int, total_count:int, interval:int):
        tracker_count += 1
        if tracker_count % int(total_count*0.05) == 0:
            print("progression : {} / {}   -   {}%".format(tracker_count, total_count, tracker_count/total_count*100))
        return tracker_count




    def train(self):
        print('Training Started\n')
        
        if self.AFFINITY_THRESHOLD == 0.0:
            raise Exception('Must calculate affinity threshold before training')
        print('affinity threshold set\n')

        print('finding the mins and maxs of each feature...')
        MINs, MAXs = self.find_MIN_MAX_dataset()
        print('min and max found\n')

        # MC initialisastion
        self.init_MC_pool()
        self.MC_iterations.append(copy.deepcopy(self.MC_POOL))


        # core training loop
        train_tracker = 0
        print('Core training loop started')
        for antigene, _class in zip(self.train_set[:,:-1],self.train_set[:,-1]):


            train_tracker = self.updateTracker(train_tracker, len(self.train_set), 5)

            # MC identification (getting the best match mc with antigene)
            if len(self.MC_POOL[_class]) == 0:
                Best_MC_match = MC(vector=antigene, _class=_class)
                self.MC_POOL[_class].append(Best_MC_match)
            else:
                Best_MC_match = self.get_max_stim_MC(_class)
            

            # ARB Generation
            self.ARB_POOL[_class].append(ARB(vector=Best_MC_match.vector, _class=_class))
            Best_MC_match_STIM = Best_MC_match.stimulate(antigene)

            iter = 0
            while True:
                iter += 1

                # generating MAX_CLONES number of ARBs from the Best_MC_match
                MAX_CLONES = int (self.CLONAL_RATE * self.HYPER_CLONAL_RATE * Best_MC_match_STIM)
                num_clones = 0
                while num_clones < MAX_CLONES:
                    newClone = Best_MC_match.mutate(MINs, MAXs, self.MUTATION_RATE)
                    self.ARB_POOL[_class].append(newClone)

                # competition for ressources
                avgStim_in_ARB_pool= sum([x.stimulate(antigene) for x in self.ARB_POOL[_class]]) / len(self.ARB_POOL[_class])
                
                MIN_STIM, MAX_STIM = self.Min_Max_Stim_ARB(antigene)

                # normalizing the ressources
                for c in self.ARB_POOL.keys():
                    for ARB_cell in self.ARB_POOL.get(c):
                        ARB_cell.stimulation = (ARB_cell.stimulation - MIN_STIM) / (MAX_STIM - MIN_STIM)
                        ARB_cell.resources = ARB_cell.stimulation * self.CLONAL_RATE

                res_allocated = sum([x.resources for x in self.ARB_POOL[_class]])
                res_allowed_limit = self.TOTAL_RESSOURCES
                while res_allocated > res_allowed_limit:
                    res_to_remove = res_allocated - res_allowed_limit
                    
                    # removing the ARB with lowest ressource
                    ARB_to_remove, ARB_to_remove_Index = self.min_res_ARB(self.ARB_POOL, _class)
                    if ARB_to_remove.ressources <= res_to_remove:
                        self.ARB_POOL[_class].remove(ARB_to_remove)
                        res_allocated -= ARB_to_remove.ressources
                    else:
                        self.ARB_POOL[_class][ARB_to_remove_Index].ressources -= res_to_remove
                        res_allocated -= res_to_remove

                if (avgStim_in_ARB_pool > self.AFFINITY_THRESHOLD) or (iter >= self.MAX_ITER):
                    break
            
            MC_candidate = self.get_max_stim_ARB_as_MC(_class)

            if MC_candidate.stimulation > Best_MC_match.stimulation:
                if self.affinity(MC_candidate.vector, Best_MC_match.vector) < self.AFFINITY_THRESHOLD * self.AFFINITY_THRESHOLD_SCALAR:
                    self.MC_POOL[_class].remove(Best_MC_match)
                MC[_class].append(MC_candidate)


            # updating the MC iteration with current pool
            self.MC_iterations.append(copy.deepcopy(MC))
        print("Training complete\n")

    





    def classify(self, antigene):
        if(self.MC_POOL is None):
            raise Exception("AIRS must be trained first")

        # stimulating all memory cells with the new antigene
        vote_array = []
        for c in self.MC_POOL.keys():
            for cell in self.MC_POOL.get(c):
                cell.stimulate(antigene)
                vote_array.append(cell)

        vote_array = np.array(list(sorted(vote_array, key=lambda cell : -cell.stimulation)))
        v = {0:0, 1:0}

        K = min(self.KNN_K, len(vote_array))

        for x in vote_array[:K]:
            v[x._class] += 1

        reverseMapping = {0: 'Benign', 1: 'Syn'}
        maxVote = 0
        _class = 0
        for x in v.keys():
            if v[x] > maxVote:
                maxVote = v[x]
                _class = x
        return _class


    def Eval(self, test_set:np.ndarray):
        print('Evaluation Started')
        n_correct= 0
        eval_tracker = 0
        lenTest = len(test_set)
        for antigene, _class in zip(test_set[:,:-1], test_set[:,-1]):
            if eval_tracker % lenTest/10 == 0:
                print('progression : {}'.format(eval_tracker/lenTest*100))
            if self.classify(antigene) == _class:
                n_correct += 1
        result = n_correct / lenTest
        print('Evaluation Finished\n')
        print('Accuracy : {}'.format(result))












class MC:
    def __init__(self, vector:np.ndarray, _class:int):
        self.vector = vector
        self._class = _class
        self.stimulation = 0.0

    def __str__(self):
        return 'MC : class = {}, stimulation = {}'.format(self._class, self.stimulation)
    def __repr__(self):
        return 'MC : class = {}, stimulation = {}'.format(self._class, self.stimulation)
    
    def stimulate(self, pattern) -> float:
        self.stimulation = AIRS.affinity(self.vector, pattern)
        return self.stimulation
    
    def mutate(self, MINs, MAXs, MUTATION_RATE):
        mutated = False
        while mutated == False:
            mutated_vect = []
            for idx, feature in enumerate(self.vector):
                if random.random() <= MUTATION_RATE:
                    mutated_vect.append(random.uniform(MINs[idx], MAXs[idx]))
                    mutated = True
                else:
                    mutated_vect.append(feature)
        return MC(vector=mutated_vect, _class=self._class)
    

class ARB:
    def __init__(self, vector, _class):
        self.vector = vector
        self._class = _class
        self.stimulation = 0.0
        self.ressources = 0

    def __str__(self):
        return 'ARB : class = {}, stimulation = {}'.format(self._class, self.stimulation)
    def __repr__(self):
        return 'ARB : class = {}, stimulation = {}'.format(self._class, self.stimulation)
    
    def stimulate(self, pattern) -> float:
        self.stimulation = AIRS.affinity(self.vector, pattern)
        return self.stimulation
    
    def mutate(self, MINs, MAXs, MUTATION_RATE):
        mutated = False
        while mutated == False:
            mutated_vect = []
            for idx, feature in enumerate(self.vector):
                if random.randome() <= MUTATION_RATE:
                    mutated_vect.append(random.uniform(MINs[idx], MAXs[idx]))
                    mutated = True
                else:
                    mutated_vect.append(feature)
        return ARB(vector=mutated_vect, _class=self._class)


In [158]:
AIRS_model = AIRS(train_set=dataset,
                  CLONAL_RATE=0.8,
                  HYPER_CLONAL_RATE=20,
                  CLASS_NUMBER=2,
                  MC_INIT_RATE=0.002,
                  MUTATION_RATE=0.5,
                  TOTAL_RESSOURCES=20,
                  MAX_ITER=1,
                  AFFINITY_THRESHOLD_SCALAR=0.8,
                  KNN_K= 7)



In [159]:
# AIRS_model.calculate_affinity_threshold()

# previously calculated  :  0.009475783090337002
AIRS_model.AFFINITY_THRESHOLD = 0.009475783090337002

In [160]:
AIRS_model.train()

Training Started

affinity threshold set

finding the mins and maxs of each feature...
min and max found

Core training loop started
progression : 3516 / 70336   -   4.9988626023657865%
progression : 7032 / 70336   -   9.997725204731573%
progression : 10548 / 70336   -   14.996587807097361%


KeyboardInterrupt: 